[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/workshops/GeoAI_Workshop_2025.ipynb)

# Object Detection from Remote Sensing Imagery with GeoAI

- 📓 **Notebook**: <https://geoai.gishub.org/workshops/GeoAI_Workshop_2025>  
- 💻 **GitHub**: <https://github.com/opengeos/geoai>

---

## 🧭 Introduction

This notebook provides hands-on materials for the workshop **Object Detection from Remote Sensing Imagery with GeoAI**, presented at the **第八届地球空间大数据与云计算研讨会** (8th Symposium on Geospatial Big Data and Cloud Computing).

## 🗂️ Agenda

The workshop will guide you through the full pipeline of GeoAI for object detection in remote sensing imagery, including:

- 📦 Package installation  
- ⬇️ Data download  
- 🖼️ Data visualization  
- 🧠 Model training  
- 🔍 Model inference  
- 🛠️ Post-processing  
- 🌐 Real-world applications  

## ⚙️ Prerequisites

- A Google Colab account (recommended for ease of setup)
- Basic familiarity with Python and geospatial data

## 📦 Package installation 

You can install the required packages using either `conda` or `pip`:

### Option 1: Using Conda (recommended for local environments)

```bash
conda create -n geoai python=3.12
conda activate geoai
conda install -c conda-forge mamba
mamba install -c conda-forge geoai
```

### Option 2: Using pip (for Colab or quick installation)

In [ ]:
# %pip install geoai-py overturemaps

## ⬇️ Data download

### Import library

In [ ]:
import geoai

### Retrieve collections

In [ ]:
collections = geoai.pc_collection_list()
collections

### Search NAIP imagery

In [ ]:
m = geoai.Map(center=[47.653010, -117.592167], zoom=16)
m

In [ ]:
bbox = m.user_roi_bounds()
if bbox is None:
    bbox = [-117.6021, 47.6502, -117.5824, 47.6559]  # Baltimore area

In [ ]:
items = geoai.pc_stac_search(
    collection="naip",
    bbox=bbox,
    time_range="2013-01-01/2024-12-31",
)

In [ ]:
items

In [ ]:
items[0]

### Visualize NAIP imagery

In [ ]:
geoai.pc_item_asset_list(items[0])

In [ ]:
geoai.view_pc_item(item=items[0])

### Download NAIP imagery

In [ ]:
downloaded = geoai.pc_stac_download(
    items[0], output_dir="data", assets=["image", "thumbnail"]
)

In [ ]:
items[0]

In [ ]:
items = geoai.pc_stac_search(
    collection="chesapeake-lc-13",
    bbox=bbox,
    time_range="2013-01-01/2014-12-31",
    max_items=10,
)

In [ ]:
items

### Search Landsat data

In [ ]:
items = geoai.pc_stac_search(
    collection="landsat-c2-l2",
    bbox=bbox,
    time_range="2023-07-01/2023-07-15",
    query={"eo:cloud_cover": {"lt": 1}},
    max_items=10,
)

In [ ]:
items

In [ ]:
items[0]

### Visualize Landsat data

In [ ]:
geoai.pc_item_asset_list(items[0])

In [ ]:
geoai.view_pc_item(item=items[0], assets=["red", "green", "blue"])

In [ ]:
geoai.view_pc_item(item=items[0], assets=["nir08", "red", "green"])

In [ ]:
geoai.view_pc_item(
    item=items[0],
    expression="(nir08-red)/(nir08+red)",
    rescale="-1,1",
    colormap_name="greens",
    name="NDVI Green",
)

### Download Landsat data

In [ ]:
geoai.pc_stac_download(
    items[0], output_dir="data", assets=["nir08", "red", "green", "blue"], max_workers=1
)

### Download building data

In [ ]:
buildings_gdf = geoai.get_overture_data(
    overture_type="building",
    bbox=bbox,
    output="data/buildings.geojson",
)

In [ ]:
buildings_gdf.head()

### Extract Building Statistics

In [ ]:
stats = geoai.extract_building_stats(buildings_gdf)
print(stats)

## Data Visualization

### Download sample datasets

In [ ]:
train_raster_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip_train.tif"
)
train_vector_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip_train_buildings.geojson"
test_raster_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip_test.tif"
)

In [ ]:
train_raster_path = geoai.download_file(train_raster_url)
train_vector_path = geoai.download_file(train_vector_url)
test_raster_path = geoai.download_file(test_raster_url)

### View metadata

In [ ]:
geoai.print_raster_info(train_raster_path, figsize=(18, 10))

In [ ]:
geoai.print_vector_info(train_vector_path, figsize=(18, 10))

### Interactive visualization

In [ ]:
geoai.view_vector_interactive(train_vector_path, tiles=train_raster_url)

In [ ]:
geoai.view_vector_interactive(
    train_vector_path,
    style_kwds={"color": "red", "fillOpacity": 0},
    tiles=train_raster_url,
)

In [ ]:
geoai.view_vector_interactive(train_vector_path, tiles="Satellite")

In [ ]:
geoai.view_raster(test_raster_url)

## 🧠 Model training

### Creating training data

In [ ]:
out_folder = "output"
tiles = geoai.export_geotiff_tiles(
    in_raster=train_raster_path,
    out_folder=out_folder,
    in_class_data=train_vector_path,
    tile_size=512,
    stride=256,
    buffer_radius=0,
)

### Train object detection model

In [ ]:
geoai.train_MaskRCNN_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/models",
    num_channels=4,
    pretrained=True,
    batch_size=4,
    num_epochs=10,
    learning_rate=0.005,
    val_split=0.2,
)

## 🔍 Model inference  

In [ ]:
masks_path = "naip_test_prediction.tif"
model_path = f"{out_folder}/models/best_model.pth"

In [ ]:
geoai.object_detection(
    test_raster_path,
    masks_path,
    model_path,
    window_size=512,
    overlap=256,
    confidence_threshold=0.5,
    batch_size=4,
    num_channels=4,
)

## 🛠️ Post-processing 

## 🌐 Real-world applications  

### Building footprint extraction

### Solar panel detection

### Car detection

### Ship detection

### Surface water mapping

### Wetland mapping